In [54]:
#Libraries
import numpy as np
from scipy.integrate import odeint

# a_i = d(v_i)/dt, where body has m_i, r_i, v_i
def acceleration(body, masses, positions):
    gravitationConstant = 6.67408e-11
    finalAcceleration = 0
    
    for j in range(masses.size):
        if body != j:
            distance = positions[j] - positions[body] # r_j - r_i
            finalAcceleration += gravitationConstant * masses[j] * distance / np.linalg.norm([distance], 2)**3
    
    return finalAcceleration


# Just Pyhton usage
def verletAlgorythm(masses, initialPosition, initialVelocity, deltaTime, iterations):
    times = np.arange(iterations) * deltaTime
    
    positions = np.zeros((times.size, masses.size))
    positions[0] = initialPosition
    
    velocities = np.zeros((times.size, masses.size))
    velocities[0] = initialVelocity
    
    currentAcceleration = np.zeros(masses.size)
    for i in range(masses.size):
        currentAcceleration[i] = acceleration(i, masses, positions[0])
        
    nextAcceleration = np.zeros(masses.size)

    for j in range(iterations-1):
        positions[j+1] = positions[j] + velocities[j] * deltaTime + 0.5 * currentAcceleration * deltaTime**2
        for i in range(masses.size):
            nextAcceleration[i] = acceleration(i, masses, positions[j+1])
        velocities[j+1] = velocities[j] + 0.5 * (currentAcceleration + nextAcceleration) * deltaTime
        currentAcceleration = nextAcceleration

    return positions, velocities, times


# odeint usage
def odeintVerletAlgorythm(masses, initialPosition, initialVelocity, deltaTime, iterations):
    times = np.arange(iterations) * deltaTime
        
    def problemPosition(v, t):
        res = np.zeros(v.size)
        for i in range(v.size):
            res[i] = v[i]
        return res
    
    def problemVelocity(x, t):
        res = np.zeros(x.size)
        for i in range(x.size):
            res[i] = acceleration(i, masses, x)
        return res

    positions = odeint(problemPosition, initialPosition, times)
    velocities = odeint(problemVelocity, initialVelocity, times)
    
    return positions, velocities, times



